# AOC 2021

Welcome to the Advent of Code 2021 !

## Basic configuration



In [ ]:
!pip install aocd

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

In [ ]:
from aocd import submit
from aocd.models import Puzzle

## Day 24
https://adventofcode.com/2021/day/24
### Part 1

### Part 2

## Day 23
https://adventofcode.com/2021/day/23
### Part 1

### Part 2

## Day 22
https://adventofcode.com/2021/day/22
### Part 1

### Part 2

## Day 21
https://adventofcode.com/2021/day/21
### Part 1

### Part 2

## Day 20
https://adventofcode.com/2021/day/20
### Part 1

### Part 2

## Day 19
https://adventofcode.com/2021/day/19
### Part 1

### Part 2

## Day 18
https://adventofcode.com/2021/day/18
### Part 1

### Part 2

## Day 17
https://adventofcode.com/2021/day/17
### Part 1

### Part 2

## Day 16
https://adventofcode.com/2021/day/16
### Part 1

### Part 2

## Day 15
https://adventofcode.com/2021/day/15

### Part 1

### Part 2

## Day 14
https://adventofcode.com/2021/day/14
### Part 1

### Part 2

## Day 13
https://adventofcode.com/2021/day/13
### Part 1

### Part 2

## Day 12
https://adventofcode.com/2021/day/12
### Part 1

### Part2

## Day 11
https://adventofcode.com/2021/day/11
### Part 1

### Part 2

## Day 10
https://adventofcode.com/2021/day/10
### Part 1

### Part 2

## Day 9
https://adventofcode.com/2021/day/9
### Part 1

### Part 2

## Day 8
https://adventofcode.com/2021/day/8
### Part 1

### Part2

## Day 7
https://adventofcode.com/2021/day/7
### Part 1

### Part 2

## Day 6
https://adventofcode.com/2021/day/6
### Part 1

### Part 2

## Day 5
https://adventofcode.com/2021/day/5
### Part 1

### Part 2

## Day 4
https://adventofcode.com/2021/day/4
### Part 1

### Part 2

## Day 3
https://adventofcode.com/2021/day/3
### Part 1

In [ ]:
puzzle = Puzzle(2021, 3)

commands = puzzle.input_data.split('\n')

In [ ]:
from collections import Counter

acc = Counter()
for cmd in commands:
    acc += Counter({idx: int(val) for idx, val in enumerate(cmd)})

In [ ]:
N, K = len(commands), len(commands[0])
gamma = '0b'+''.join(str(int(acc[i] > N/2)) for i in range(K))
gamma = int(gamma, 2)
epsilon = (~gamma & 0xFFF)
answer_a = gamma*epsilon

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def filter_cmds(commands, pos, lcb):
    acc = sum(int(cmd[pos]) for cmd in commands)
    mc = (acc >= len(commands) / 2)
    if lcb:
        mc = not mc
    mc = int(mc)
    return [cmd for cmd in commands if int(cmd[pos]) == mc]

def apply(cmds, lcb=False):
    for pos in range(K):
        cmds = filter_cmds(cmds, pos, lcb)
        if len(cmds) == 1:
            break
    return cmds

oxy = int(apply(commands.copy(), lcb=False)[0], 2)
co2 = int(apply(commands.copy(), lcb=True)[0], 2)

answer_b = oxy*co2

In [ ]:
puzzle.answer_b = answer_b

## Day 2
https://adventofcode.com/2021/day/2
### Part 1

In [ ]:
from collections import Counter

puzzle = Puzzle(2021, 2)

commands = puzzle.input_data.split('\n')

global_cmd = Counter()

for cmd in commands:
    word, amount = cmd.split()
    global_cmd[word] += int(amount)
    
answer_a = global_cmd['forward']*(global_cmd['down'] - global_cmd['up'])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
status = Counter()

for cmd in commands:
    word, amount = cmd.split()
    amount = int(amount)
    if word == "forward":
        status['forward'] += amount
        status['depth'] += amount*status['aim']
    elif word == "up":
        status['aim'] -= amount
    else:
        status['aim'] += amount
        
answer_b = status["forward"]*status["depth"]

In [ ]:
puzzle.answer_b = answer_b

## Day 1
https://adventofcode.com/2021/day/1
### Part 1

In [ ]:
puzzle = Puzzle(2021, 1)

depths = [int(depth) for depth in puzzle.input_data.split()]
answer_a = len([d1 for d1, d2 in zip(depths[:-1], depths[1:]) if d2 > d1])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
moving_average = [d1+d2+d3 for d1, d2, d3 in zip(depths[:-2], depths[1:-1], depths[2:])]
answer_b = len([d1 for d1, d2 in zip(moving_average[:-1], moving_average[1:]) if d2 > d1])

In [ ]:
puzzle.answer_b = answer_b